In [1]:
import os
import gc
import pickle
import pandas as pd
import numpy as np
import sys
from os import listdir
from os.path import isfile, join
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_recall_fscore_support as score
import multiprocessing
from joblib import Parallel, delayed

path                 = "."

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# FIX 1 of 3 : set the following vars
path                 = "." #path to dir where test pickles reside and h5 file is
modelfingerprint     = "kubernetes_train_[80-20]_giga_long_mltithrdedgentr_PikTranSav_Mdl_[J_Filternet_tstep1]_epch_75_kube_binar_crsentr_adm_binar_acc.py" # name of py file
modelweightsfilename = "kubernetes_train_[80-20]_giga_long_mltithrdedgentr_PikTranSav_Mdl_[J_Filternet_tstep1]_epch_75_kube_binar_crsentr_adm_binar_acc.h5" # name of h5 file

# FIX 2 of 3: update the input_length as per timesteps
input_length    = 1 # 1 or 2                        # X_final.shape[1]
input_dim       = 3541                              # X_final.shape[2]
output_dim      = 1

pickgpus = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = pickgpus

def create_model(input_dim, input_length, output_dim):
    print ('Creating model...')
    # FIX 3 of 3: copy everything from here till 'return parallel_model'
    model = Sequential()
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(1000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000, input_shape=(input_length,input_dim),return_sequences=True))
    model.add(LSTM(2000))
    model.add(Dense(output_dim, activation='sigmoid'))
    ###
    print('Initiating parallel GPU model')
    parallel_model = multi_gpu_model(model, gpus=1+pickgpus.count(","))
    print ('Compiling...')
    parallel_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['binary_accuracy'])
    return parallel_model

In [3]:
# STEP 1
model   = create_model(input_dim, input_length, output_dim)
# STEP 2
print('Loading Model weights before Testing')
model.load_weights( join(path, modelweightsfilename) )

Creating model...
Initiating parallel GPU model
Compiling...
Loading Model weights before Testing


In [4]:
# warm up
warmup=True

In [5]:
test_files = [f for f in listdir(path) if isfile(join(path, f))]
test_files = [x for x in test_files if 'dict_jobID_' in x and 'FROM_giga_takeoutdf_w_jobgroup.pickle' in x ]
test_files.sort()

In [6]:
import itertools

def chunked(it, size):
    it = iter(it)
    while True:
        p = tuple(itertools.islice(it, size))
        if not p:
            break
        yield p

In [7]:
# parameters
from threading import Lock

samplefrac         = 0.1
decision_threshold = 0.95
minsamplestopick   = 3
lock               = Lock()

def h(jobID_Xtest_Ytest):
    global model
    global samplefrac
    global global_dict
    global lock
    global takeoutdf
    global minsamplestopick
    
    localsamplefrac = samplefrac
    Y_true = []
    Y_pred = []
    #
    #
    
    jobID, X_test, Y_test     = jobID_Xtest_Ytest[0], jobID_Xtest_Ytest[1][0], jobID_Xtest_Ytest[1][1] 
    
    # model is global
        
    try:
        lock.acquire()
        y_pred_this_file = model.predict(X_test)
    finally:
        lock.release()
    
    Y_pred.extend([ np.rint(jj[0]) for jj in y_pred_this_file ])
    Y_true.extend([ xj[0] for xj in Y_test ])
    
    #test 1
    if not all(x == Y_true[0] for x in Y_true):
        print('All Y_true labels for jobID %d are not same \n' % jobID)

    truelabel = Y_true[0]
    
    # perform job-level prediction in a while loop:
    firstN         = min( max(minsamplestopick, int(localsamplefrac * len(Y_true))), len(Y_pred) )
    predictedlabel = -1
    
    while(predictedlabel == -1 and localsamplefrac <= 1.0):
        localsamplefrac = (firstN*1.0) / len(Y_pred)
        take            = Y_pred[:firstN]
        
        if take.count(1.0) > decision_threshold * len(take):
            predictedlabel = 1
            break
            
        if take.count(0.0) > decision_threshold * len(take):
            predictedlabel = 0
            break
        
        firstN          = firstN + 1
        
    
    # create a python list that will be return
    # send back
    totaljobduration   = None
    predictionduration = None
     
    return [jobID, totaljobduration, predictionduration, localsamplefrac, len(Y_true), decision_threshold, truelabel, predictedlabel]

In [8]:
global_results = []
import random

for k in range(len(test_files)):

    # Load { jobID -> X_test, Y_test }:
    print( "Loading test file #%d - %s" % (k, test_files[k]) )
    with open( join(path, test_files[k]) , 'rb' ) as handle:
        global_dict = pickle.load(handle)
    print( "Loaded test file #%d" % (len(global_dict))       )
    
    # warm up
    if(warmup):
        print(model.predict(random.choice(list(global_dict.items()))[1][0]))
        warmup=False
        
    start = 1
    for chunk in chunked(global_dict.items(), 48*5):
        print("New Batch started : %d to %d " % (start, start+len(chunk)-1))
        results = Parallel(n_jobs=48, verbose=1, backend="threading")(map(delayed(h), [x for x in chunk] ))
        global_results.extend(results)
        del results
        start += len(chunk)

Loading test file #0 - dict_jobID_Xtest_Ytest_0_32072_FROM_giga_takeoutdf_w_jobgroup.pickle
Loaded test file #32073
[[6.75055338e-03]
 [5.38457790e-03]
 [6.37775147e-03]
 [6.15844456e-03]
 [5.54621033e-03]
 [4.54523368e-03]
 [2.87896115e-03]
 [1.72375410e-03]
 [1.16244704e-03]
 [8.33725906e-04]
 [6.72767812e-04]
 [4.61185176e-04]
 [4.13235161e-04]
 [3.23646236e-04]
 [2.76857842e-04]
 [2.02255906e-04]
 [1.01623154e-04]
 [4.75871202e-05]]
New Batch started : 1 to 240 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    5.5s
[Parallel(n_jobs=48)]: Done 240 out of 240 | elapsed:   12.6s finished


New Batch started : 241 to 480 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    5.5s
[Parallel(n_jobs=48)]: Done 240 out of 240 | elapsed:   12.7s finished


New Batch started : 481 to 720 


...

[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    5.4s
[Parallel(n_jobs=48)]: Done 240 out of 240 | elapsed:   12.4s finished


New Batch started : 31681 to 31920 


[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    5.4s
[Parallel(n_jobs=48)]: Done 240 out of 240 | elapsed:   12.5s finished


New Batch started : 31921 to 32073 


[Parallel(n_jobs=48)]: Done 153 out of 153 | elapsed:    7.9s finished


# Considering all jobs where decision was made

In [9]:
# Process results:

Y_true_job      = []
Y_predicted_job = []
averagefrac     = []
nodecision      = []

for r in global_results:
    if r[-1] != -1:
        Y_true_job.append(r[-2])
        Y_predicted_job.append(r[-1])
        averagefrac.append(r[3])
    else:
        nodecision.append(r)

# [jobID, totaljobduration, predictionduration, localsamplefrac, len(df), decision_threshold, truelabel, predictedlabel]

In [10]:
print('Len of Y_true : ', len(Y_true_job))
print('Len of Y_pred : ', len(Y_predicted_job))

print("Overall accuracy on test set: ", np.mean(np.equal(Y_true_job, Y_predicted_job)))

print("Calculating Score: ")
precision, recall, fscore, support = score(Y_true_job, Y_predicted_job)
print('precision    : {}'.format(precision))
print('recall       : {}'.format(recall))
print('fscore       : {}'.format(fscore))
print('support      : {}'.format(support))

print("Confusion matrix:")
print(pd.crosstab(pd.Series(Y_true_job), pd.Series(Y_predicted_job), rownames=['True'], colnames=['Predicted'], margins=True))
print('*********Test Results End**************')

Len of Y_true :  310423
Len of Y_pred :  310423
Overall accuracy on test set:  0.8491735470632008
Calculating Score: 
precision    : [0.82965447 0.87542586]
recall       : [0.89957127 0.79257542]
fscore       : [0.86319942 0.83194303]
support      : [164206 146217]
Confusion matrix:
Predicted       0       1     All
True                             
0          147715   16491  164206
1           30329  115888  146217
All        178044  132379  310423
*********Test Results End**************


In [11]:
np.mean(averagefrac)

0.5467337450251508

In [16]:
len(nodecision), len(Y_true_job)+len(nodecision),  len(nodecision)/len(Y_true_job)+len(nodecision)

(10307, 320730, 10307.033203080957)

# Considering jobs with atleast 4 samples

In [35]:
# Process results:

Y_true_job      = []
Y_predicted_job = []
averagefrac     = []
nodecision      = []

for r in global_results:
    if r[-1] != -1:
        if(r[4] >= 10):
            Y_true_job.append(r[-2])
            Y_predicted_job.append(r[-1])
            averagefrac.append(r[3])
    else:
        nodecision.append(r)

# [jobID, totaljobduration, predictionduration, localsamplefrac, len(df), decision_threshold, truelabel, predictedlabel]

In [36]:
print('Len of Y_true : ', len(Y_true_job))
print('Len of Y_pred : ', len(Y_predicted_job))

print("Overall accuracy on test set: ", np.mean(np.equal(Y_true_job, Y_predicted_job)))

print("Calculating Score: ")
precision, recall, fscore, support = score(Y_true_job, Y_predicted_job)
print('precision    : {}'.format(precision))
print('recall       : {}'.format(recall))
print('fscore       : {}'.format(fscore))
print('support      : {}'.format(support))

print("Confusion matrix:")
print(pd.crosstab(pd.Series(Y_true_job), pd.Series(Y_predicted_job), rownames=['True'], colnames=['Predicted'], margins=True))
print('*********Test Results End**************')

Len of Y_true :  133237
Len of Y_pred :  133237
Overall accuracy on test set:  0.8343403108746069
Calculating Score: 
precision    : [0.88916507 0.71147437]
recall       : [0.87352044 0.74122507]
fscore       : [0.88127333 0.72604508]
support      : [93778 39459]
Confusion matrix:
Predicted      0      1     All
True                           
0          81917  11861   93778
1          10211  29248   39459
All        92128  41109  133237
*********Test Results End**************


In [37]:
np.mean(averagefrac)

0.18735241246471637

In [34]:
len(nodecision), len(Y_true_job)+len(nodecision),  len(nodecision)/len(Y_true_job)+len(nodecision)

(10307, 106072, 10307.107628047825)